# Исследование надежности заемщиков.

   В настоящее время широкое распростронение получили модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. Для обучения таких систем используют статистику по клиентам, с которой нам с вами сегодня предстоит работать. 
    
    
**Целью** этого проекта является определение влияния данных о клиенте на факт погашения кредита в срок.
    
**Задачи:**
 1. Определить есть ли зависимость между количеством детей и возвратом кредита в срок?
 2. Определить есть ли зависимость между семейным положением и возвратом кредита в срок?
 3. Определить есть ли зависимость между уровнем дохода и возвратом кредита в срок
 4. Определить как разные цели кредита влияют на его возврат в срок?

**Объектом** исследования выступают клиенты банка.

**Предмет исследования:** определение влияния данных о клиенте на факт погашения кредита в срок.

**Методы** которые мы применим в данном исследовании включают в себя: сравнение, предобработку, анализ и категоризацию данных о заемщиках.

**Структкра проекта.** Проектная работа состоит из обзора и предобработки данных, затем следует категоризация данных, подготовка таблиц для решения поставленных задач и наконец заключительная часть с выводами и заключением.  



### Шаг 1. Обзор данных

Откроем таблицу и ознакомимся с данными:

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('/datasets/data.csv')

# Выведем на экран первые 10 строк DataFrame
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Изучим общую информацию о данных:

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Мы обнаружили пропуски данных в колонках 'days_employed' и 'total_income', также 
примем во внимание что доля пропусков в этих колонках одинакова.

Возможные причины появления пропусков в данных:
1. Ошибки при выгрузке из БД
2. Человеческий фактор (клиенты намеренно могут не указать данные)

Хотя, навряд ли клиенты банка не указали свой доход при заполнении заявки на кредит здесь скорее всего мы имеем дело с первой причиной.


### Шаг 2.1 Заполнение пропусков

Вычислим какую долю составляют пропущенные значения 
от столбца с пропусками:

In [3]:
data.isna().mean().sort_values(ascending=False)

days_employed       0.100999
total_income        0.100999
children            0.000000
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
purpose             0.000000
dtype: float64

10% данных в столбце 'total_income' утеряны, эти данные являются количественным значениям, пропуски в количественных переменных заполняют характерными значениями. Это значения, характеризующие диапазон набора данных, выбранных для проведения исследования. Чтобы примерно оценить типичные значения выборки, применяются среднее арифметическое или медиану.

Заполним значение столбца 'total_income' медианным значением с учетом категории 'income_type' чтобы сильно не усреднять данные:

In [4]:
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))

Проверим столбец 'total_income' на отсутствующие значения:

In [5]:
data['total_income'].isna().mean()

0.0

Теперь заполним пропуски в данные столбца 'days_employed' медианным значением:

In [6]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean())

Работа с пропущенными значениями завершена, еще раз проверим количество пропусков:

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [8]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.673959e+05
std,1.381587,133526.013419,12.574584,0.548138,1.420324,0.272661,9.790695e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-982.531720,42.000000,1.000000,0.000000,0.000000,1.519313e+05
75%,1.000000,63046.497661,53.000000,1.000000,1.000000,0.000000,2.024175e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В столбцах данных 'children' и 'days_employed' имеются отрицательные значения.
К тому же в столбце 'days_employed' четверть выборки имеет завышенное, неадекватное значение, в реальной жизни пришлось бы уточнять у ответственного за выгрузку выборки. 
Скорее всего это технологическая ошибка.
Заменим отрицательные значения в столбце 'days_employed' использую фунцию abs():

In [9]:
data['days_employed'] = abs(data['days_employed'])

In [10]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Значения столбца 'education' являются категоральной переменной записанной с
разным регистром, причина появления разных значений человеческий фактор, а 
именно отсутсвие стандартизации при вводе данных клиентом, исправим это:

In [11]:
data['education'] = data['education'].str.lower()

В столбце 'children' обнаружили отрицательное значение количества детей, исправим это

In [12]:
data['children'].unique()



array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [13]:
data['children'] = abs(data['children'])
data['children'].unique()


array([ 1,  0,  3,  2,  4, 20,  5])

В 76 строках DataFrame клиенты указали что имеют 20 детей:

In [14]:
data[data['children'] == 20]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,63046.497661,53,среднее,1,женат / замужем,0,M,компаньон,0,202417.461462,покупка жилой недвижимости
21404,20,494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


Наше исследование от их потери не слишком измениться, и мы не знаем, как эти данные логично заменить, поэтому заменим их на 0:

In [15]:
data['children'] = data['children'].replace(20, 0)


### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце 'total_income' на целочисленный:

In [16]:
data.astype({'total_income': 'int64'}).dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
dtype: object

### Шаг 2.4. Удаление дубликатов.

Найдем дублирующиеся строки DataFrame и удалим их, также сбрасываем индекс:

In [17]:
data.duplicated().sum()

71

In [18]:
data = data.drop_duplicates().reset_index(drop=True)

Снова проверим DataFrame на повторяющиеся строки:

In [19]:
data.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [20]:
# Сгруппируем наш DataFrame по уникальным значениюм столбца 'education',
# в качестве столбца данных выберем 'education_id'
unique_education = data.groupby(data['education'])['education_id'].unique()  

# Создадим из сгруппированного объекта DataFrame в котором каждому
#  уникальному значению из education соответствует уникальное 
#  значение education_id, также сбрасываем индекс для выравнивания 
#  заголовков колонок и изменяем тип данных колонки 'education_id'
# на числовой
unique_education = (pd.DataFrame(unique_education)
.reset_index().astype({'education_id': 'int'}))

unique_education

,education,education_id
0,высшее,0
1,начальное,3
2,неоконченное высшее,2
3,среднее,1
4,ученая степень,4


In [21]:
# Сгруппируем наш DataFrame по уникальным значениюм столбца 'family_status',
# в качестве столбца данных выберем 'family_status_id'
unique_family_status = data.groupby(data['family_status'])['family_status_id'].unique()

# Создадим из сгруппированного объекта DataFrame в котором каждому
#  уникальному значению из 'family_status' соответствует уникальное 
#  значение 'family_status_id', также сбрасываем индекс для выравнивания 
#  заголовков колонок и изменяем тип данных колонки 'education_id'
# на числовой
unique_family_status = (pd.DataFrame(unique_family_status)
.reset_index().astype({'family_status_id': 'int'}))

unique_family_status


,family_status,family_status_id
0,Не женат / не замужем,4
1,в разводе,3
2,вдовец / вдова,2
3,гражданский брак,1
4,женат / замужем,0


Удалим столбцы 'education' и 'family_status' из DataFrame:

In [22]:
data.drop(columns = ['education', 'family_status'],axis = 1)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,1,1,F,компаньон,0,224791.862382,операции с жильем
21450,0,343937.404131,67,1,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21451,1,2113.346888,38,1,1,M,сотрудник,1,89672.561153,недвижимость
21452,3,3112.481705,38,1,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

В столбце DataFrame 'total_income_category' указан месячный доход клиентов, множество значений
категоризируем на следующие категории:
1. 'E' 0 - 30 000
2. 'D' 30 001 - 50 000
3. 'C' 50 001 - 20 0000
4. 'B' 20 0001 - 10 00000
5. 'A' свыше 1 000 000

In [23]:
# Объявляем фунцию "total_income_category" которая на основании параметра row будет принимать
# строки DataFrame в качестве результата возвращает категорию заработка клиента 
def total_income_category(row):
    if 0 <= row['total_income'] <= 30000:
        return 'E'
    elif 30000 < row['total_income'] <= 50000:
        return 'D'
    elif 50000 < row['total_income'] <= 200000:
        return 'C'
    elif 200000 < row['total_income'] <= 1000000:
        return 'B'
    elif 1000000 < row['total_income']: 
        return 'A'
    
# Применим фунцию "total_income_category" к нашему DataFrame в качестве
# аргумента axis зададим построчное применение
data['total_income_category'] = data.apply(total_income_category, axis=1)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,B
21450,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,C
21451,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,C
21452,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,B


### Шаг 2.7. Категоризация целей кредита.

В столбце DataFrame 'purpose' указанны цели взятия кредита, множество значений
категоризируем на четыре категории:
1. Операции с автомобилем
2. Операции с недвижимостью
3. Проведение свадьбы
4. Получение образования

In [24]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [25]:
# Объявляем фунцию 'total_purpose_category' которая на основании параметра row будет принимать
# строки DataFrame в качестве результата возвращает категорию целевого использования кредитных
# средств

def total_purpose_category(row):
    if 'автом' in row['purpose']:
        return 'операции с автомобилем'
    elif 'недвиж' in row['purpose'] or 'жиль' in row['purpose']:
        return 'операции с недвижимостью'
    elif 'свадь' in row['purpose']:
        return 'проведение свадьбы'
    elif 'образ' in row['purpose']:
        return 'получение образования'

In [26]:
data['purpose_category'] = data.apply(total_purpose_category, axis=1) # Применим фунцию 'total_purpose_category' к нашему DataFrame в качестве
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

### Ответы на вопросы.

##### Вопрос 1:

Для ответа на вопрос: 'Есть ли зависимость между количеством детей и возвратом кредита в срок?' создадим таблицу, в которой количество детей будут являться категориями; кол-во клиентов в категории, кол-во просрочивших оплату клиентов и доля просрочивших клиентов - колонками таблицы, соответсвенно. Также отсортируем данные по убыванию доли просрочивщих оплату клиентов:

In [27]:
 # Создание DataFrame
debt_from_children = pd.DataFrame()  # Объявляем переменную debt_from_children - новый DataFrame

 # Добавляем колонки в DataFrame 
debt_from_children['count_children'] = data.groupby('children')['debt'].count() 
debt_from_children['sum_children'] = data.groupby('children')['debt'].sum()
debt_from_children['result_children'] = (debt_from_children['sum_children'] 
                                        / debt_from_children['count_children'])
    
 # Сортировка, форматирование, переименование колонок и индекса  
debt_from_children.index = debt_from_children.index.rename('Кол-во детей')
debt_from_children.rename(columns={'count_children': 'Клиентов в категории',
                                   'sum_children': 'Просрочившие',
                                   'result_children': 'Доля просрочивших'}, 
                          inplace=True)
debt_from_children.reset_index(inplace=True, drop=False)
debt_from_children = debt_from_children.sort_values('Доля просрочивших', ascending=False)

debt_from_children

,Кол-во детей,Клиентов в категории,Просрочившие,Доля просрочивших
4,4,41,4,0.097561
2,2,2052,194,0.094542
1,1,4855,445,0.091658
3,3,330,27,0.081818
0,0,14167,1071,0.075598
5,5,9,0,0.000000


##### Вывод 1:

Зависимость между количеством детей и возвратом кредита в срок есть. 7.5 % клиентов не имеющие детей реже всех остальных категорий просрочивают оплату по кредитам. Затем идет тенденция к увеличению до 9 % просрочивших оплату клиентов в категориях имеющих от 1-го до 2-х детей. У клиентов с 3-мя детьми показатель просрочки начинает уменьшаться до 8 %. Данные выборки не однозначные, скорее всего требуется более обширная выборка.

##### Вопрос 2:

In [28]:
def create_table(target_value, data): # Объявляем функцию 'create_table' которой в качестве параметров 
    # передаем целевое значние и DataFrame, фунция возвращает таблицу для анализа целевого показателя по
    # категориям целевого значения
    
    table = pd.pivot_table(data, index=[target_value], # Создание сводной таблицы
                                         values=['debt'], 
                                         aggfunc=[len, sum, np.mean])
    
    table.columns = ['_'.join(col).strip() for col in table.columns.values] # Преобразование 
    # мультииндекса в одноуровневый индекс в наименованиях столбцов DataFrame
    
    table.reset_index(inplace=True, drop=False) # Вынесение индекса в отдельную колонку для выравнивания заголовков
    
    table = table.rename(columns={'len_debt': 'Нет просрочек', # Переименование столбцов 
                                  'sum_debt': 'Просрочившие',
                                  'mean_debt': 'Доля просрочивших'})
    return table   

Для ответа на вопрос: 'Есть ли зависимость между семейным положением и возвратом кредита в срок?' создадим таблицу, в которой семейное положение будет являться категориями; кол-во клиентов в категории, кол-во просрочивших оплату клиентов и доля просрочивших клиентов - колонками таблицы, соответсвенно. Также отсортируем данные по убыванию доли просрочивщих оплату клиентов:

In [29]:
debt_from_family_status = create_table('family_status', data)
debt_from_family_status = debt_from_family_status.rename(columns={'family_status': 'Семейное положение'})
debt_from_family_status = debt_from_family_status.sort_values('Доля просрочивших', ascending=False)

debt_from_family_status

,Семейное положение,Нет просрочек,Просрочившие,Доля просрочивших
0,Не женат / не замужем,2810,274,0.097509
3,гражданский брак,4151,388,0.093471
4,женат / замужем,12339,931,0.075452
1,в разводе,1195,85,0.071130
2,вдовец / вдова,959,63,0.065693


##### Вывод 2:

Зависимость между семейным положением и возвратом кредита в срок есть. Клиенты не состоящие в официальном браке чаще остальных просрочиваю оплату 9.3 - 9.7 %.
Мы видим уменьшение на 1.8 - 2.2% доли просрочивших оплату в категориях: женатых и разведенных, и заметное уменьшени доли просрочивших в категории 'вдовец / вдова' до 6.5 %.

##### Вопрос 3:

Для ответа на вопрос: 'Есть ли зависимость между уровнем дохода и возвратом кредита в срок?' создадим таблицу, в которой ранее разбитые категории уровня дохода клиентов будет являться категориями; кол-во клиентов в категории, кол-во просрочивших оплату клиентов и доля просрочивших клиентов - колонками таблицы, соответственно. Также отсортируем данные возростанию уровня доходов клиентов:

In [30]:
debt_from_income_category = create_table('total_income_category', data) 
debt_from_income_category = debt_from_income_category.rename(columns={'total_income_category': 'Уровень доходов'})
debt_from_income_category = debt_from_income_category.sort_values('Уровень доходов', ascending=False)
debt_from_income_category

,Уровень доходов,Нет просрочек,Просрочившие,Доля просрочивших
4,E,22,2,0.090909
3,D,350,21,0.060000
2,C,15514,1330,0.085729
1,B,5543,386,0.069637
0,A,25,2,0.080000


##### Вывод 3:

Зависимости между уровнем дохода и возвратом кредита в срок не наблюдается, данные выборки не однозначны, требуется более обширная выборка по категориям дохода.

##### Вопрос 4:

Для ответа на вопрос: 'Есть ли зависимость между целями кредитования и возвратом кредита в срок?' создадим таблицу, в которой цели кредитования будет являться категориями; кол-во клиентов в категории, кол-во просрочивших оплату клиентов и доля просрочивших клиентов - колонками таблицы, соответственно. Также отсортируем данные по убыванию доли просрочивщих оплату клиентов:

In [31]:
debt_from_purpose_category = create_table('purpose_category', data)    
debt_from_purpose_category = debt_from_purpose_category.rename(columns={'purpose_category': 'Цель кредитования'})
debt_from_purpose_category = debt_from_purpose_category.sort_values('Доля просрочивших', ascending=False)
debt_from_purpose_category


,Цель кредитования,Нет просрочек,Просрочившие,Доля просрочивших
0,операции с автомобилем,4306,403,0.093590
2,получение образования,4013,370,0.092200
3,проведение свадьбы,2324,186,0.080034
1,операции с недвижимостью,10811,782,0.072334


##### Вывод 4:

Зависимость между целями кредита и его возврат в срок есть. Операции с автомобилем и получения образования имеют наиболее высокую долю просрочивших кредит 9.3 и 9.2 процентов соответсвенно. Клиенты взявшие кредит проведения свадьбы просрочивают оплату в 8 % случаев. Самые надежные клиенты берут кредит на операции с недвижимостью, просрочка в этой категории всего 7 %.

## Общий вывод:

 В ходе проекта мы ознакомились с данными, обработали пустые значения с учетом категорий, избавились от дублирующих записей. Затем стандартизировали типы данных и значения для категоризации по целям кредитования и категориям дохода. Для решения поставленных задач и ответов на вопросы, мы построили таблицы зависимости целевого показателя 'debt' (факт просрочки по кредиту) от: количества детей, семейного положения, категории дохода и целей кредитования.
    
 Общий вывод - чем больше детей, тем чаще платят не в срок. Бездетные реже просрачивают оплату. Семейное положение влияет на вероятность просрочки по кредиту - люди, бывшие в браке чаще плятят в срок чем те, кто не бывали в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. Автокредиты и получение образования - цели по которым чаще наблюдаются просрочки, на проведение свадеб и на операции с недвижимостью - просрочивают платежи реже.

Привет), Приятно познакомиться, внес правки по твоим замечаниям, спасибо за комментарии!